In [52]:
import pandas as pd

fs_path = r'C:\Users\rkddk\OneDrive\문서\카카오톡 받은 파일\perdata.xlsx'
invest_df=pd.read_excel(fs_path)


# 종목코드가 6자리가 되도록 앞에 0을 채우는 함수
def pad_stock_code(df, column_name):
    df[column_name] = df[column_name].astype(str).str.zfill(6)
    return df

# 데이터프레임에 종목코드 6자리 맞추기 적용
invest_df = pad_stock_code(invest_df, '종목코드')

# PER 기준으로 오름차순으로 정렬하여 주는 함수
def low_per(invest_df, index_date, num):
    invest_df = invest_df[invest_df['기준일'] == index_date]  # 날짜 필터링
    invest_df['PER'] = pd.to_numeric(invest_df['PER'], errors='coerce')  # PER 컬럼을 숫자로 변환
    per_sorted = invest_df.sort_values(by='PER')  # PER 기준으로 정렬
    return per_sorted[:num] if num else per_sorted  # 상위 num개의 행을 반환 (num이 None이면 전체 반환)

# ROA 기준으로 내림차순으로 정렬하여 주는 함|수
def high_roa(invest_df, index_date, num):
    invest_df = invest_df[invest_df['기준일'] == index_date]  # 날짜 필터링
    invest_df['ROA'] = pd.to_numeric(invest_df['ROA'], errors='coerce')  # ROA 컬럼을 숫자로 변환
    sorted_roa = invest_df.sort_values(by='ROA', ascending=False)  # ROA 기준으로 내림차순 정렬
    return sorted_roa[:num] if num else sorted_roa  # 상위 num개의 행을 반환 (num이 None이면 전체 반환)

# per, roa 정렬한 데이터프레임을 가져오기
date = '2022/12'
per = low_per(invest_df, date, None)
roa = high_roa(invest_df, date, None)

# 결과 출력
print("PER 기준 오름차순 정렬:")
print(per)
print("\nROA 기준 내림차순 정렬:")
print(roa)


PER 기준 오름차순 정렬:
       종목코드      기준일      PER     PCR   PSR   PBR  영업이익률   ROA
0    011200  2022/12     0.95    0.87  0.51  0.46   53.5  46.1
1    016380  2022/12     1.38    1.28  0.19  0.44    8.9  18.4
2    010060  2022/12     1.74    1.49  0.55  0.39   27.2  14.1
3    078930  2022/12     1.94    1.94  0.15  0.33   17.9   7.6
58   000880  2022/12     2.14    0.99  0.05  0.23    4.7   1.0
..      ...      ...      ...     ...   ...   ...    ...   ...
43   271940  2022/12   153.90   86.83  9.94  3.44    2.5   2.1
70   018880  2022/12   211.29    7.47  0.50  1.93    3.0   0.3
129  336260  2022/12   621.73  148.30  7.70  4.59    2.3   0.5
42   112610  2022/12  1521.13   42.99  2.11  3.15    3.1  -0.4
87   009420  2022/12  3697.74  281.43  8.45  5.12    1.4   0.1

[153 rows x 8 columns]

ROA 기준 내림차순 정렬:
       종목코드      기준일      PER     PCR   PSR   PBR  영업이익률   ROA
0    011200  2022/12     0.95    0.87  0.51  0.46   53.5  46.1
21   383220  2022/12    12.53   10.60  3.06  5.88   29.0  32.

In [55]:
per['per순위']=per['PER'].rank()
per['roa순위'] = per['ROA'].rank(ascending=False)
per['마법공식 순위'] = (per['per순위'] + per['roa순위']).rank().sort_values()
per=per.sort_values(by='마법공식 순위')
per

,기준일,PER,PCR,PSR,PBR,영업이익률,ROA,per순위,roa순위,마법공식 순위
종목코드,,,,,,,,,,
011200,2022/12,0.95,0.87,0.51,0.46,53.5,46.1,1.0,1.0,1.0
016380,2022/12,1.38,1.28,0.19,0.44,8.9,18.4,2.0,5.5,2.0
010060,2022/12,1.74,1.49,0.55,0.39,27.2,14.1,3.0,9.0,3.0
000990,2022/12,2.97,2.45,0.99,1.00,45.6,30.1,11.0,3.0,4.0
111770,2022/12,3.08,2.71,0.53,0.74,21.0,18.4,13.0,5.5,5.0
...,...,...,...,...,...,...,...,...,...,...
011170,2022/12,103.25,6.30,0.29,0.44,-3.4,0.1,142.0,150.5,149.0
336260,2022/12,621.73,148.30,7.70,4.59,2.3,0.5,151.0,144.0,150.0
018880,2022/12,211.29,7.47,0.50,1.93,3.0,0.3,150.0,147.0,151.0


In [57]:
#함수로 정리
def magic_formula(invest_df, index_date, num):
    per = low_per(invest_df, index_date, None)
    per['per순위'] = per['PER'].rank()
    per['roa순위'] = per['ROA'].rank(ascending=False)
    per['마법공식 순위'] = (per['per순위'] + per['roa순위']).rank().sort_values()
    per=per.sort_values(by='마법공식 순위')
    return per[:num]